# $addFields 的操作

In [ ]:
# reassign ``gravity`` field value
db.solarSystem.aggregate([
    {
        $project: {
            "gravity": "$gravity.value"
        }
    }

]).pretty()

In [ ]:
# adding ``name`` and removing ``_id`` from projection
db.solarSystem.aggregate([
    {
        $project: {
            _id: 0,
            name: 1,
            "gravity": "$gravity.value"
        }
    }

]).pretty()

In [ ]:
# adding more fields to the projected document
db.solarSystem.aggregate([
    {
        $project: {

            _id: 0,
            name: 1,
            gravity: "$gravity.value",
            meanTemperature: 1,
            density: 1,
            mass: "$mass.value",
            radius: "$radius.value",
            sma: "$sma.value"
        }
    }
]).pretty()

In [ ]:
# using ``$addFields`` to generate the new computed field values
db.solarSystem.aggregate([
    {
        $addFields: {
            gravity: "$gravity.value",
            mass: "$mass.value",
            radius: "$radius.value",
            sma: "$sma.value"
        }
    }
]).pretty()

In [ ]:
# combining ``$project`` with ``$addFields``
db.solarSystem.aggregate([
    {
        $project: {
            _id: 0,
            name: 1,
            gravity: 1,
            mass: 1,
            radius: 1,
            sma: 1
        }
    },
    {
        $addFields: {

            gravity: "$gravity.value",
            mass: "$mass.value",
            radius: "$radius.value",
            sma: "$sma.value"
        }
    }
]).pretty()

# $geoNear (aggregation) 的操作

In [ ]:
# using ``$geoNear`` stage
db.nycFacilities.aggregate([
    {
        $geoNear: {
            near: {
                type: "Point",
                "coordinates": [-73.98769766092299, 40.757345233626594]
            },
            distanceField: "distanceFromMongoDB",
            spherical: true
        }
    }
]).pretty()

In [ ]:
# 加入更多 Fields
# 離 mongoDB 總部最近的五間醫院
db.nycFacilities.aggregate([
    {
        $geoNear: {
            near: {
                type: "Point",
                "coordinates": [-73.98769766092299, 40.757345233626594]
            },
            distanceField: "distanceFromMongoDB",
            spherical: true,
            query: { type: "Hospital" },
        }
    },
    {
        $limit: 5
    }
]).pretty()

# $limit 的操作

# $skip 的操作

# $sort 的操作

In [ ]:
# project fields ``numberOfMoons`` and ``name``
db.solarSystem.find({}, {"_id": 0, "name": 1, "numberOfMoons": 1}).pretty()

# count the number of documents
db.solarSystem.find({}, {"_id": 0, "name": 1, "numberOfMoons": 1}).count()

# skip documents
db.solarSystem.find({}, {"_id": 0, "name": 1, "numberOfMoons": 1}).skip(5).pretty()

# limit documents
db.solarSystem.find({}, {"_id": 0, "name": 1, "numberOfMoons": 1}).limit(5).pretty()

# sort documents
db.solarSystem.find({}, { "_id": 0, "name": 1, "numberOfMoons": 1 }).sort( {"numberOfMoons": -1 } ).pretty()

In [ ]:
# ``$limit`` stage
db.solarSystem.aggregate([
    {
        $project: {

            _id: 0,
            name: 1,
            numberOfMoons: 1
        }
    },
    { $limit: 5}
]).pretty()

In [ ]:
# ``skip`` stage
db.solarSystem.aggregate([
    {
        $project: {

            _id: 0,
            name: 1,
            numberOfMoons: 1
        }
    },
    { $skip: 1 }
]).pretty()

In [ ]:
# ``$count`` stage
db.solarSystem.aggregate([
    {
        $match: {
            type: "Terrestrial planet"
        }
    },
    {
        $project: {
            _id: 0,
            name: 1,
            numberOfMoons: 1
        }
    },
    {
        $count: "terrestrial planets"
    }
]).pretty()

In [ ]:
# removing ``$project`` stage since it does not interfere with our count
db.solarSystem.aggregate([
    {
        $match: {
            type: "Terrestrial planet"
        }
    },
    {
        $count: "terrestrial planets"
    }
]).pretty()

In [ ]:
# ``$sort`` stage
db.solarSystem.aggregate([
    {
        $project: {

            _id: 0,
            name: 1,
            numberOfMoons: 1
        }
    },
    { $sort: { numberOfMoons: -1 } }
]).pretty()

In [ ]:
# sorting on more than one field
db.solarSystem.aggregate([
    {
        $project: {

            _id: 0,
            name: 1,
            numberOfMoons: 1,
            hasMagneticField: 1
        }
    },
    { $sort: { hasMagneticField: -1, numberOfMoons: -1 } }
]).pretty()

In [ ]:
# setting ``allowDiskUse`` option
db.solarSystem.aggregate([
    {
        $project: {

            _id: 0,
            name: 1,
            numberOfMoons: 1,
            hasMagneticField: 1
        }
    },
    { $sort: { hasMagneticField: -1, numberOfMoons: -1 } }
], {allowDiskUse: true }).pretty()

# $sample 的操作

In [ ]:
# sampling 200 documents of collection ``nycFacilities``
db.nycFacilities.aggregate([
    {
        $sample: { size: 200 }
    }
]).pretty()

### 範例01

In [ ]:
var favorites = [
    "Sandra Bullock",
    "Tom Hanks",
    "Julia Roberts",
    "Kevin Spacey",
    "George Clooney"]

db.movies.aggregate([
    {
        $match: {
            countries: { $in: [ "USA" ] },
            "tomatoes.viewer.rating": { $gte: 3 },
            cast: { $in: favorites }
        }
    },
    {
        $project: {
            _id: 0,
            title: 1,
            tomatoes: "$tomatoes.viewer.rating",
            num_favs: { $size: { $setIntersection: [ "$cast", favorites ] } }
        }
    },
    {
        $sort: {
            num_favs: -1,
            tomatoes: -1,
            title: -1
        }
    },
    { $skip: 24 },
    { $limit: 1}
]).pretty()

### 範例02

In [ ]:
# 這兩個數字怎麼求，下一章會教
x_max = 1521105
x_min = 5

db.movies.aggregate([
    {
        $match: {
            languages: { $in: ["English" ] },
            "imdb.rating": { $gte: 1 },
            "imdb.votes": { $gte: 1},
            year: { $gte: 1990 }
        }
    },
    {
        $project: {
            _id: 0,
            title: 1,
            imdbRating: "$imdb.rating",
            imdbVotes: { $add: [ 1, { $multiply: [ 9, { $divide: [ { $subtract: [ "$imdb.votes", x_min] }, { $subtract: [ x_max, x_min ]  } ] } ] } ]}
        }
    },
    {
        $project: {
            title: 1,
            normalized_rating: { $avg: [ "$imdbRating", "$imdbVotes" ]}
        }
    },
    { $sort: { normalized_rating: 1 }},
    { $limit: 1 }

]).pretty()